In [3]:
import pylab as pl
import numpy as np
import pandas 
import regressData as data
import ridgeRegression as rr

1. Implement ridge regression (see Bishop equation 3.27, 3.28); the closed-form solution for the optimal solution of ridge regression is easy to calculate so do not use gradient descent for 3.1 and 3.2.  Use the polynomial basis function for 3.1 and 3.2. Experiment with different values of $\lambda$ on the simple data from the previous question (curvefittingp2.txt), for various values of $M$. Describe your observations.

    - BISHOP 3.27: $\frac{1}{2} \sum_{n = 1}^{N} \{t_n - w^{T}\phi(x_n)\}^2 + \frac{\lambda}{2}w^{T}w$ 

    - BISHOP 3.28: $w = (\lambda I + \phi^{T}\phi)^{-1}\phi^{T}t$

